In [1]:
# Install Required Libraries
!pip install --q sentence-transformers==2.2.2 langchain==0.0.179 chromadb==0.3.25 typing-inspect==0.8.0 typing_extensions==4.5.0

# Import Required Libraries
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from torch.utils.data import DataLoader
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.7/907.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.4/964.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
# Download the Embedding Model
original_model = SentenceTransformer('all-MiniLM-L12-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
!pip install --q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql import functions as fn

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("ProductSearchDataPrep").getOrCreate()

# Set the path for the dataset files
#downloads_path = "/path/to/your/downloads"

In [ ]:
# Step 1: Read and Write Products Data
products_schema = StructType([
    StructField('product_id', IntegerType()),
    StructField('product_name', StringType()),
    StructField('product_class', StringType()),
    StructField('category_hierarchy', StringType()),
    StructField('product_description', StringType()),
    StructField('product_features', StringType()),
    StructField('rating_count', FloatType()),
    StructField('average_rating', FloatType()),
    StructField('review_count', FloatType())
])

products_df = (
    spark
    .read
    .csv('/content/drive/MyDrive/WANDS/dataset/product.csv', sep='\t', header=True, schema=products_schema)
)

products_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/products')

# Step 2: Read and Write Queries Data
queries_schema = StructType([
    StructField('query_id', IntegerType()),
    StructField('query', StringType()),
    StructField('query_class', StringType())
])

queries_df = (
    spark.read
    .csv('/content/drive/MyDrive/WANDS/dataset/query.csv', sep='\t', header=True, schema=queries_schema)
)

queries_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/queries')

# Step 3: Read and Write Labels Data
labels_schema = StructType([
    StructField('id', IntegerType()),
    StructField('query_id', IntegerType()),
    StructField('product_id', IntegerType()),
    StructField('label', StringType())
])

labels_df = (
    spark.read
    .csv('/content/drive/MyDrive/WANDS/dataset/label.csv', sep='\t', header=True, schema=labels_schema)
)

labels_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/labels')

# Step 4: Add Label Score Column to Labels Table
#labels_df = spark.read.csv('/content/drive/MyDrive/WANDS/dataset/label.csv', header=True, inferSchema=True)

if "label_score" not in labels_df.columns:
    labels_df = labels_df.withColumn("label_score",
                                     fn.when(labels_df["label"] == "Exact", 1.0)
                                     .when(labels_df["label"] == "Partial", 0.75)
                                     .when(labels_df["label"] == "Irrelevant", 0.0)
                                     .otherwise(None))

labels_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/labels_with_score')


In [ ]:
queries_df.show()

+--------+--------------------+--------------------+
|query_id|               query|         query_class|
+--------+--------------------+--------------------+
|       0|         salon chair|      Massage Chairs|
|       1|  smart coffee table|Coffee & Cocktail...|
|       2|            dinosaur|     Kids Wall Décor|
|       3|   turquoise pillows|      Accent Pillows|
|       4|chair and a half ...|           Recliners|
|       5|   sofa with ottoman|          Sectionals|
|       6| acrylic clear chair|       Dining Chairs|
|       7|    driftwood mirror|Wall & Accent Mir...|
|       8|home sweet home sign|          Wall Décor|
|       9|coffee table fire...|  Outdoor Fireplaces|
|      10|     king poster bed|                Beds|
|      11|           ombre rug|           Area Rugs|
|      12|large spoon and f...|          Wall Décor|
|      13|outdoor privacy wall|Fencing & Accesso...|
|      14| beds that have leds|                Beds|
|      15|black 5 drawer dr...|   Dressers & C

In [ ]:
import pandas as pd
import os

# Specify the folder containing the CSV files
folder_path = '/content/drive/MyDrive/WANDS/dataset/labels'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Check if there are at least two CSV files
if len(csv_files) >= 2:
    # Read each CSV file into a separate DataFrame
    dfs = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

    # Concatenate the DataFrames vertically
    combined_df = pd.concat(dfs, ignore_index=True)

    # Write the combined DataFrame to a new CSV file
    combined_df.to_csv(os.path.join(folder_path, 'combined_labels.csv'), index=False)
    print("CSV files combined successfully.")
else:
    print("There are not enough CSV files to combine.")


CSV files combined successfully.


In [ ]:
import pandas as pd
import os

# Specify the folder containing the CSV files
folder_path = '/content/drive/MyDrive/WANDS/dataset/products'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Check if there are at least two CSV files
if len(csv_files) >= 2:
    # Read each CSV file into a separate DataFrame
    dfs = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]

    # Concatenate the DataFrames vertically
    combined_df = pd.concat(dfs, ignore_index=True)

    # Write the combined DataFrame to a new CSV file
    combined_df.to_csv(os.path.join(folder_path, 'combined_products.csv'), index=False)
    print("CSV files combined successfully.")
else:
    print("There are not enough CSV files to combine.")


CSV files combined successfully.


In [ ]:
# Get Search Results
search_df = queries_df  # Replace with the path to your search data CSV

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Assuming you have a SparkSession named 'spark' and a DataFrame named 'products_df'
# with columns 'product_description' and 'product_name'

# Create a new column 'product_text' with the coalesced values
products_df = products_df.withColumn(
    'product_text',
    fn.coalesce('product_description', 'product_name')
)

# If you want to replace NaN values in multiple columns, you can use the following approach
# Specify the list of columns to be coalesced
columns_to_coalesce = ['product_description', 'product_name']

# Create a new column 'product_text' with the coalesced values from specified columns
products_df = products_df.withColumn(
    'product_text',
    fn.coalesce(*columns_to_coalesce)
)

# Now, 'products_df' contains the new 'product_text' column with coalesced values


In [ ]:
products_df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------+------------+--------------------+
|product_id|        product_name|       product_class|  category_hierarchy| product_description|    product_features|rating_count|average_rating|review_count|        product_text|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------+------------+--------------------+
|         0|solid wood platfo...|                Beds|Furniture / Bedro...|good , deep sleep...|overallwidth-side...|        15.0|           4.5|        15.0|good , deep sleep...|
|         1|all-clad 7 qt . s...|        Slow Cookers|Kitchen & Tableto...|create delicious ...|capacityquarts:7|...|       100.0|           2.0|        98.0|create delicious ...|
|         2|all-clad electric...|        Slow Cookers|Kitchen & Tableto...|prepare home-cook...|feat

In [ ]:
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="query", outputCol="words")
queries_df = tokenizer.transform(queries_df)

# Step 5: Tokenize Products
tokenizer = Tokenizer(inputCol="product_text", outputCol="words")
products_df = tokenizer.transform(products_df)

# Define the Word2Vec model
word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="words", outputCol="embeddings")

# Create a pipeline with the tokenizer and Word2Vec model
pipeline = Pipeline(stages=[tokenizer, word2Vec])

# Fit the pipeline to the queries data and transform it to get query embeddings
query_embeddings_model = pipeline.fit(queries_df)
query_embeddings_df = query_embeddings_model.transform(queries_df)

# Fit the pipeline to the products data and transform it to get product embeddings
product_embeddings_model = pipeline.fit(products_df)
product_embeddings_df = product_embeddings_model.transform(products_df)

# Save the DataFrames with embeddings to CSV
query_embeddings_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/query_embeddings')
product_embeddings_df.write.mode("overwrite").csv('/content/drive/MyDrive/WANDS/dataset/product_embeddings')


IllegalArgumentException: ignored

In [ ]:
queries_df

DataFrame[query_id: int, query: string, query_class: string, words: array<string>]

In [ ]:
# Convert Queries & Products to Embeddings
query_embeddings = original_model.encode(search_df['query'].tolist())
product_embeddings = original_model.encode(search_df['product_text'].tolist())

TypeError: ignored

In [ ]:
# Calculate Cosine Similarity Between Queries and Products
original_cos_sim_scores = util.pairwise_cos_sim(query_embeddings, product_embeddings)

In [ ]:
# Calculate Avg Cosine Similarity
original_cos_sim_score = torch.mean(original_cos_sim_scores).item()

In [ ]:
# Calculate Correlation with Scores
original_corr_coef_score = np.corrcoef(original_cos_sim_scores, search_df['score'].values)[0][1]

In [ ]:
# Fine-Tune the Model
tuned_model = SentenceTransformer('all-MiniLM-L12-v2')
input_dataloader = DataLoader([InputExample(texts=[row['query'], row['product_text']], label=row['score']) for _, row in search_df.iterrows()], shuffle=True, batch_size=16)
loss = losses.CosineSimilarityLoss(tuned_model)
tuned_model.fit(train_objectives=[(input_dataloader, loss)], epochs=1, warmup_steps=100)

In [ ]:
# Estimate Fine-Tuned Model Performance
query_embeddings = tuned_model.encode(search_df['query'].tolist())
product_embeddings = tuned_model.encode(search_df['product_text'].tolist())
tuned_cos_sim_scores = util.pairwise_cos_sim(query_embeddings, product_embeddings)

In [ ]:
# Calculate Avg Cosine Similarity
tuned_cos_sim_score = torch.mean(tuned_cos_sim_scores).item()

In [ ]:
# Calculate Correlation Coefficient
tuned_corr_coef_score = np.corrcoef(tuned_cos_sim_scores, search_df['score'].values)[0][1]

In [ ]:
# Persist Model for Deployment
product_text_df = pd.read_csv('path/to/your/product_text_data.csv')  # Replace with the path to your product text data CSV
documents = DataFrameLoader(product_text_df, page_content_column='product_text').load()

In [ ]:
embedding_model_path = "/path/to/your/tuned_model"
tuned_model.save(embedding_model_path)
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_path)

In [ ]:
chromadb_path = "/path/to/your/tuned_chromadb"
vectordb = Chroma.from_documents(documents=documents, embedding=embedding_model, persist_directory=chromadb_path)
vectordb.persist()

In [ ]:
# Define Wrapper Class for Model
class ProductSearchWrapper:
    def __init__(self, vectordb, max_results=5):
        self._vectordb = vectordb
        self._max_results = max_results

    def predict(self, query):
        raw_results = self._vectordb.similarity_search_with_score(query, k=self._max_results)
        scores, descriptions, names, ids = zip(*[(r[1], r[0].page_content, r[0].metadata['product_name'], r[0].metadata['product_id']) for r in raw_results])
        results_df = pd.DataFrame({
            'product_id': ids,
            'product_name': names,
            'product_description': descriptions,
            'score': scores
        }).sort_values(axis=0, by='score', ascending=True)
        return results_df

In [ ]:
with open('/path/to/your/model_wrapper.pkl', 'wb') as f:
    pickle.dump(wrapper, f)

In [ ]:
# Load the Wrapper Class and Test the Model
with open('/path/to/your/model_wrapper.pkl', 'rb') as f:
    loaded_wrapper = pickle.load(f)

In [ ]:
search_query = 'farmhouse dining room table'
result = loaded_wrapper.predict(search_query)
print(result)